In [ ]:
#|default_exp concurrent.remote.test_ProxyStateHandler_0

In [ ]:
#|export
import asyncio, os, sys, subprocess
from multiprocessing.connection import Listener
from fbdev._utils import TaskManager, find_available_port
from fbdev.dev_utils import is_in_repl
from fbdev.concurrent.remote import RemoteController, get_client, ProxyStateHandler, ProxyStateHandlerMediator

In [ ]:
#|export
port_num = find_available_port()

remote_py_script = """
address = ('localhost', %s)
authkey = b'secret'

import asyncio, os, multiprocessing
from multiprocessing.connection import Listener
from fbdev._utils import TaskManager, StateHandler
from fbdev.concurrent.remote import RemoteController, ProxyStateHandler, ProxyStateHandlerMediator

listener = Listener(address, authkey=authkey)
async def main(conn):
    task_manager = TaskManager(None)
    remote = RemoteController(conn, task_manager)
    
    my_state_local = StateHandler('my_state', True)
    my_state_mediator = ProxyStateHandlerMediator('main', remote, task_manager, my_state_local)
    
    print('Child: Setting `my_state=True`')
    my_state_local.set(True) # Setting before remote.await_ready()
    
    remote.send_ready()
    await remote.await_ready()
    
    print('Child: Awaiting `my_state=False`')
    await my_state_local.wait(False)
    print(f'Child: `my_state={my_state_local.get()}`')
    assert not my_state_local.get()
    
    await asyncio.sleep(1)
    
with listener.accept() as conn:
    asyncio.run(main(conn))
""" % port_num

address = ('localhost', port_num)
authkey = b'secret'

async def test():
    task_manager = TaskManager(None)
    proc = subprocess.Popen([sys.executable, '-c', remote_py_script])

    with get_client(address, authkey=authkey) as conn:
        remote = RemoteController(conn, task_manager)
        
        proxy_state_handler = ProxyStateHandler('my_state', parent_handle='main', remote=remote)
        
        remote.send_ready()
        await remote.await_ready()
        await proxy_state_handler.await_initialised()
        
        print('Parent: Awaiting `my_state=True`')
        await proxy_state_handler.get_state_toggle_event().wait()
        print(f'Parent: `my_state={proxy_state_handler.get()}`')
        assert proxy_state_handler.get()
        
        print('Parent: Setting `my_state=False`')
        proxy_state_handler.set(False)
        
        await asyncio.sleep(1)
        
    await task_manager.destroy()
    proc.communicate()  # This will wait until the process completes

In [ ]:
await test()

Child: Setting `my_state=True`
Child: Awaiting `my_state=False`
Parent: Awaiting `my_state=True`
Parent: `my_state=True`
Parent: Setting `my_state=False`
Child: `my_state=False`
